Things to try to increase performance: 
-even more data
-separate models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis
-Look for transformations opps, etc
-model only using genre

In [1]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

TEMP_FOLDER = tempfile.gettempdir()

In [2]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [3]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [4]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [5]:
links[5] = text

In [6]:
links.drop_duplicates([5], inplace = True)


In [7]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [8]:
links.shape

(2903, 6)

In [9]:
pd.set_option('display.max_rows', 500)
grouped = links.groupby([3]).count()
grouped.sort_values(by=[2],ascending = False, inplace = True)
cols = grouped[:14].index.get_level_values(0)
colslist = cols.tolist()
selected = links[links[3].isin(colslist)]
selected.shape

(2503, 6)

In [10]:
selected.head()

,0,1,2,3,4,5
0,https://www.ign.com/articles/2018/11/17/battle...,7.5,PC,Shooter,"November 17, 2018","[""With all of the development muscle behind EA..."
1,https://www.ign.com/articles/2018/11/15/sid-me...,9.4,Nintendo Switch,Strategy,"November 15, 2018",['When I first reviewed Civilization VI back i...
2,https://www.ign.com/articles/2018/11/14/warfra...,8.6,PC,Shooter,"November 15, 2018","[' ', 'There aren’t many games that make me fe..."
3,https://www.ign.com/articles/2018/11/14/overki...,5.3,PC,Action,"November 14, 2018","['Overkill’s ', ' is an earnest attempt to del..."
4,https://www.ign.com/articles/2018/11/16/pokemo...,8.3,Nintendo Switch,RPG,"November 12, 2018","[""It's often with rose-colored glasses that I ..."


In [11]:
selected.groupby([3]).mean()

,1
3,
Action,6.977104
"Action, Adventure",7.644262
"Action, RPG",7.656790
Adventure,7.359877
Fighting,7.556627
Hardware,8.450000
Platformer,7.629341
Puzzle,7.615909
RPG,7.545263


In [12]:
#any nan?
selected.isna().sum()


0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [13]:
selected.drop([0,5,4], axis = 1, inplace = True)
selected.shape

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(2503, 3)

In [14]:
selected = pd.get_dummies(selected)
selected.shape

(2503, 39)

In [15]:
#train val split

trainval = selected.sample(600)
train = selected.loc[~selected.index.isin(trainval.index), :]
y = train[1]
valy = trainval[1]
trainval.drop([1], axis = 1, inplace = True)
train.drop([1], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## XGBoost

In [ ]:
#bayesian opt
yarray = y.values
dtrain = xgb.DMatrix(all_topics_numpy, label=yarray)
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7),
                                            'gamma': (0, 1),
                                            'colsample_bytree': (0.3, 0.9)})

xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')



In [357]:
param = xgb_bo.max
param

{'target': -1.1006746666666667,
 'params': {'colsample_bytree': 0.8976729774092942,
  'gamma': 0.9971962663965198,
  'max_depth': 4.321612175930944}}

In [99]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(train,yarray)

predictions2 = xgbreg.predict(trainval)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.3891798123147852

## Lasso Regression

In [ ]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

In [ ]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(all_topics_numpy,yarray)

predictions3 = lasso.predict(all_topics_numpytest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


## OLS

10.0

In [77]:
yarray = y.values
lm = LinearRegression()
lm.fit(train,yarray)
predictions = lm.predict(trainval)
predictionsdf = pd.DataFrame(predictions)

In [82]:
predictionsdf.describe()

,0
count,6.000000e+02
mean,9.236435e+11
std,1.420428e+13
min,5.843750e+00
25%,7.093750e+00
50%,7.359375e+00
75%,7.625000e+00
max,2.409915e+14


In [ ]:
valy = valy.reset_index()
valy

In [ ]:
valy.drop(['index'], axis = 1, inplace = True)
valy

In [ ]:
predictionsdf['true'] = valy
predictionsdf

In [ ]:
predictionsdf = predictionsdf.sort_values(by=[0], ascending = False)
predictionsdf = predictionsdf.iloc[3:]
predictionsdf

In [ ]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

In [333]:
#let's also plot histogram of scores




In [334]:
#residual plot here




In [97]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.398608940028378

In [ ]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff